# LiricAI

Aqui é mostrado um pequeno projecto em que se usa uma Recurrent Neural Network (segundo https://gilberttanner.com/blog/generating-text-using-a-recurrent-neuralnetwork) para criar a letra de uma música a partir de um determinado conjunto de letras já existente. 

## Os dados

Os ficheiros utilizados encontram-se disponíveis no repositório de Github como ficheiros .txt

## Bibliotecas a usar

Como estou a seguir a ligação acima referida, Keras e Tensorflow.

## Tratamento de dados

In [9]:
from os import listdir
from os.path import isfile, join
import sys
import collections, functools, operator
import numpy as np
import random
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM
from keras.optimizers import RMSprop
from keras.callbacks import LambdaCallback, ModelCheckpoint, ReduceLROnPlateau

Contar os caracteres presentes nas letras:

In [10]:
dataset_path = "./dataset"
files_in_dataset = [f for f in listdir(dataset_path) if isfile(join(dataset_path, f))]
char_indices_list = []
indices_char_list = []

total_lyrics = ""
for filename in files_in_dataset:
    filepath = dataset_path +"/"+filename
    with open(filepath, 'r',encoding='utf8') as file:

        text = file.read().lower()
      
        total_lyrics += "\n\n"+text 

chars = sorted(list(set(total_lyrics))) # getting all unique chars

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

print(char_indices)

{'\n': 0, ' ': 1, '!': 2, '"': 3, "'": 4, '(': 5, ')': 6, '+': 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '6': 17, '7': 18, '8': 19, '9': 20, ':': 21, ';': 22, '?': 23, '[': 24, ']': 25, 'a': 26, 'b': 27, 'c': 28, 'd': 29, 'e': 30, 'f': 31, 'g': 32, 'h': 33, 'i': 34, 'j': 35, 'k': 36, 'l': 37, 'm': 38, 'n': 39, 'o': 40, 'p': 41, 'q': 42, 'r': 43, 's': 44, 't': 45, 'u': 46, 'v': 47, 'w': 48, 'x': 49, 'y': 50, 'z': 51, '{': 52, '}': 53, '«': 54, '´': 55, '»': 56, 'à': 57, 'á': 58, 'â': 59, 'ã': 60, 'ç': 61, 'é': 62, 'ê': 63, 'í': 64, 'ó': 65, 'ô': 66, 'õ': 67, 'ú': 68, '\u2005': 69, '‘': 70, '’': 71, '“': 72, '”': 73, '…': 74, '\u205f': 75}


In [11]:
max_len = 40
step = 3
sentences = []
next_chars = []

for i in range(0, len(total_lyrics) - max_len, step):
    sentences.append(total_lyrics[i: i + max_len])
    next_chars.append(total_lyrics[i + max_len])

x = np.zeros((len(sentences), max_len, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

## Recurrent Neural Network

In [12]:
model = Sequential()
model.add(LSTM(128, input_shape=(max_len, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [13]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - max_len - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + max_len]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, max_len, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [14]:
filepath = "weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss',
                             verbose=1, save_best_only=True,
                             mode='min')

In [15]:
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=1, min_lr=0.001)

callbacks = [print_callback, checkpoint, reduce_lr]

## Treinar a rede

In [ ]:
model.fit(x, y, batch_size=128, epochs=64, callbacks=callbacks)

Epoch 1/64
456/457 [============================>.] - ETA: 0s - loss: 2.2888
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "bate a cabeça no teto
e cai na travessa
"
bate a cabeça no teto
e cai na travessa

a morreitar

e eu se me me corrar
se me me me me corina

e eu se me me prepor

e a merinha de corter

e eu se precera

e eu se me me me prepar

e eu se me me corina

não se prepreiar

e eu se me me corto
e eu se pera em se preira

não so te corar

e para entera a minha me cortar

e ventro o corre a me amor

e eu se pera no corrar

a marinha em sober

não sor se me me corter

e prastira a minha ve
----- diversity: 0.5
----- Generating with seed: "bate a cabeça no teto
e cai na travessa
"
bate a cabeça no teto
e cai na travessa

no degre pora perura

a que eu se um corato

não me te corir

e a vortando o corero 
a mer
não sa prodia

não ter pera eu cordo

vonto laninho éu teu so prara
de é corder
ao merires

não coriro

e for a areirar

e eu so 

não te acartar de amor de amor é tão nos sons
não há que a las não amor
pra mas se a canção

se tu desta com a piraver em trou

e a canta de olhar o sol assim por me amar

mãos que sentir o salo a contar 
e
----- diversity: 1.0
----- Generating with seed: "a
onde o espaço é todo meu
não são obras"
a
onde o espaço é todo meu
não são obras que há que enrondece

outo que quero etabre agor de mamor 
e ir sempre pra oscora o que agor de márea astroval
e eu bando, ilham nos mara
outra premente
de vamiaa

o que fendefio viso-te a entrirvas
estava mas deixo que fui a tu dan)

lá mais os laso a alhar de ti
ela mápica foi nada ruar de te
abras
na seve no vonto com o meu calco os colaç
mes farto porque um tr canção
e erante mesla pra trzgan
----- diversity: 1.2
----- Generating with seed: "a
onde o espaço é todo meu
não são obras"
a
onde o espaço é todo meu
não são obras fil que as mãos esse
êlhão, outouena, nós coração
ão antuluisa,s só nomor que algongião com esperar-me alguém os pemar
regrenhó

do tudo ser como tesso porfou

comoja devazer o vil-te com rnto

na nova naprem tadoressa homa , comacana, cantas, pacar, matar
nós case te umar
se tu estás já foreu poder os lichá z
re)
o tenho
que este olhar de quem amonte
à noite é o braço quando o que pras cansal se essa cano
apana cá vida acordos
consego

Epoch 00007: loss improved from 1.42146 to 1.36647, saving model to weights.hdf5
457/457 [==============================] - 66s 144ms/step - loss: 1.3665
Epoch 8/64
457/457 [==============================] - ETA: 0s - loss: 1.3230
----- Generating text after Epoch: 7
----- diversity: 0.2
----- Generating with seed: "scurso
citando o sujeito
primeira pessoa"
scurso
citando o sujeito
primeira pessoa a cordena
não contanto a mano
e a culva pera consia
e a culpe a carrensa
para ser um pazer a carra
talvez se falas

eu não se o mundo eu amor

aquila por mim de mar
e se a marte
na terra mas sem resasso

e eu sentir o sol

sentir o sol

sentir o sol)
sentir o sol)
sentir o sol)
sentir o

estava e sem ter a dar
não sei o que eu não sei 
estava escura de mim

estava e não te ser que estava esperto
esta voz não sei se estar a
----- diversity: 0.5
----- Generating with seed: "ossos
pois só sei crescer

eu só me caib"
ossos
pois só sei crescer

eu só me caibo de fortar
vem tem contar ilharessa, teu salho do amorece
e o mundo a tua condeçue
e eu amor traberta
que seres a marte

espara sempre a alguém da como a carregar
se se o meu andochão
e nem sei que a merenci
para só tenho o prelecite

tontes eu não sei o que me vou a parar

perde de ver e queres emerito

mais ue meus espriar

estou de te mais que eu sou eu só

e que leva a carrente para te não qu
----- diversity: 1.0
----- Generating with seed: "ossos
pois só sei crescer

eu só me caib"
ossos
pois só sei crescer

eu só me caibo a tração
dexportasso mais vendo ao praves
tondecenda a olha dela é de manhã
bamífazesse da lembra intentra a melho

na virque freio sem baixa na minha espade
ter tropo para alencimal

de ser preg

se ser te ejreves tor marcam
que tu veste mas vais tucas

desle bassa pra hillá para tem, a corperte de talcas 
ao latido de paixão no meu el....(
se um dia eu não sei quem for
a vida é sempre noite
o contror, forte de lutar

já não ver d
----- diversity: 1.2
----- Generating with seed: "as bato no peito
por estar com meu ar ra"
as bato no peito
por estar com meu ar razão

que ham esque, não tea é comeda
denira-te da cida comé
joglo um assoas frinhos lacotel
se é que quando canina
aqumla aste vou ele pansão eu besto
o céu naviana ztal

ai não comobo tem bastogoí
aósgim te ao c,lpeco....

e quando a dóriada e marrine
tanta fazia de ti,
e não quise
mas o que eu quero certo
só amágis-me sonhadaves à tua murcadir
desvamos soleino amor atér

juntir te luto armempeir

Epoch 00014: loss improved from 1.19209 to 1.17823, saving model to weights.hdf5
457/457 [==============================] - 67s 147ms/step - loss: 1.1782
Epoch 15/64
457/457 [==============================] - ETA: 0s - loss: 1.

...

457/457 [==============================] - ETA: 0s - loss: 0.9674
----- Generating text after Epoch: 34
----- diversity: 0.2
----- Generating with seed: "invento virtude
no pico não peco
lá em b"
invento virtude
no pico não peco
lá em briar

não vai tua mais um dia que me vou estar a carte
a pensar asse eu sentir à solta do chão
dormim a sous sentir à minha dizer de ti

a carte se eu fui nada mais de ti

a nos fina lá carta

estava a sentida

se o mundo éde vou amor

e o mundo é uma lutança 
e o muhto que não faz chegar a sua menta
não vou dorda o meu amor
que me os marisfosinhos
do brico do céu, mimhar de mar

se a senho ter de
----- diversity: 0.5
----- Generating with seed: "invento virtude
no pico não peco
lá em b"
invento virtude
no pico não peco
lá em briar

nas minhas da visviagias, as fricamos

aquecebo para trocado

se eu sei que não mais
ou meus que não mais temprento
ela se descar a sestida em chapasrar

se a luz da vir de ser fertir
vai ser um dia que te digas casas
e as ma

C:\Users\danie\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


 agirar a chuva do vera

se está quando eu sou fortentes
não te conseguem alcanter

as mantas da ver de que te estar a dia

eu não te o mundo ficaregar

eu sei que não vou deito a ti
e tu aqui
e quis tanto f
----- diversity: 0.5
----- Generating with seed: " sei inventar-me
é só mais do mesmo
ferm"
 sei inventar-me
é só mais do mesmo
ferm, muito de ver

se estám-te sou eu não te estás
sinha mãe cedo

só estar a de que te esperar
e a praz de contar, você tensa esta canção
quando o jeito
fica até amar

estava encomo a cidra e compreia

se um dia e o sepantas jogar no az

não umas nunca está pra norte consim

e se eu nunca minha mais dia
e nunca meira sabe a vida é frego
e deres ao chão

se um destio por amarrui
se o mundo forar todo
----- diversity: 1.0
----- Generating with seed: " sei inventar-me
é só mais do mesmo
ferm"
 sei inventar-me
é só mais do mesmo
fermos das esplezo
que sou-te ao pé de danhadeira avantelia
e não ver o meio do que nem ver

apé de vou eu passavo a voz carra
apre

as uma, sespaça, as nungados da canção
do pé de ti
e era senta a mar

enquas no estoca a dero uma correnção
de brianme e me salta esperar
dos teus olhos morinheiro à seteteira
vão presceta, ver o meu caminha
que há de sentir o soljos
do aquino de horar

do que ser viver que o freitoce o que eu pa
chama

acolver 
----- diversity: 1.2
----- Generating with seed: "eito

eu nem cá dentro me caibo
pois bat"
eito

eu nem cá dentro me caibo
pois baterreãó
junto vpi
há uqueci nunca sela

ninguém e adá acordo cedo que como  sega mansida
não é ção é o condias
não é do jardim
o mundo intemo te que no puatia
o pauco chiroe!
nisses já feipácom nos fim
se estar pedinges quem começo
que enlhece e me quer minha rafha
te me luz doisbe e abatar

ele foi tentar à traça de erábatoras
eles mogad....

até que mo é à falmar
em vistas atrazer pertão
de mansi

Epoch 00041: loss improved from 0.93417 to 0.93145, saving model to weights.hdf5
457/457 [==============================] - 66s 144ms/step - loss: 0.931

Epoch 45/64
456/457 [============================>.] - ETA: 0s - loss: 0.8975
----- Generating text after Epoch: 44
----- diversity: 0.2
----- Generating with seed: "to no peito
por estar com meu ar rarefei"
to no peito
por estar com meu ar rarefeito
e a carnei de ti esta canção
que a carne se feresa de ser

vou dar, dar, dar, dar
dar e receber

eu sei de te querer maixo de mais
quero ser prima tamoce para te olhar
quero ser quem tempos
eu não sei

quero sobreste a dia em que sou estrete
tenho uma muas que uma lua
e que lição nos dás por não responderes

e o mundo é de maré
o que é que queres sentir o sol
o meu fim como eu amor a vida

eu n
----- diversity: 0.5
----- Generating with seed: "to no peito
por estar com meu ar rarefei"
to no peito
por estar com meu ar rarefeito
e a chuva de sonhar que te levam a gartedo

mas se é o despoja no fundo de mar
esta carãe de mar
num ninhe fica de fogo

ela não se assim é vocebo é meu
não sou eu, não sou eu, miúda?
miúda, que não vou deira
vem hor

In [ ]:
def generate_text(length, diversity):
    # Get random starting text
    start_index = random.randint(0, len(text) - max_len - 1)
    generated = ''
    sentence = text[start_index: start_index + max_len]
    generated += sentence
    for i in range(length):
            x_pred = np.zeros((1, max_len, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char
    return generated

In [ ]:
import datetime
number_of_files = 10

date = datetime.datetime.now().date()

for counter in range(0, number_of_files):
    songFileName = "results/output_"+str(date)+"_"+str(counter)+".txt"
    with open(songFileName, 'w') as f:
        f.write(generate_text(500, 0.2))

#print(generate_text(500, 0.2))